<a href="https://colab.research.google.com/github/MarogobaCTD/Gemini/blob/main/Projeto_Imersao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets

In [ ]:
#Importação das bibliotecas e autenticação
import os
import io
import pandas as pd
import numpy as np
import ipywidgets as widgets
import google.generativeai as genai
from IPython.display import display
from google.colab import userdata
from google.colab import output
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

lista_arquivo = {}

In [ ]:
# Informações para o modelo
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 2048,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [ ]:
def upload_arquivo():

  def on_button_clicked(change):
      try:
          #recuperando as informações do arquivo
          arquivo = uploaded_file.value[0]
          #print(arquivo)
          chave_nome = arquivo['name']
          #lista_arquivo.pop(chave_nome)
          total = len(arquivo['content'])
          #verificando se já existe um arquivo com o mesmo nome
          #e gravar um seguencial
          nome_arquivo = pega_arquivo(chave_nome)
          if (nome_arquivo != chave_nome):
              print('Saved {antigo} to {novo}'.format(
                  antigo=chave_nome, novo=os.getcwd() + '/' + nome_arquivo))
          #print('passou')
          #preparando a progressão para agravação do arquivo
          prog = widgets.IntProgress(value=0,
                                      min=0,
                                      max=total,
                                      description = 'Carregando...',
                                      bar_style='success',
                                      orientation='horizontal')
          display(prog)

          #Criando o arquivo localmente
          with open(nome_arquivo, 'wb') as f:
            for i in range(0, len(arquivo['content']), 1024):
              # Obtenha um pedaço do conteúdo
              data = arquivo['content'][i:i+1024]
              f.write(data)
              prog.value += len(data)
          iniciar_chat()
      except Exception as e:
          print(e)

  def pega_arquivo(nomeArquivo):
      if not os.path.lexists(nomeArquivo):
          return nomeArquivo
      counter = 1
      while True:
          path, ext = os.path.splitext(nomeArquivo)
          novo_nome = '{} ({}){}'.format(path, counter, ext)
          if not os.path.lexists(novo_nome):
              return novo_nome
          counter += 1

  # Determinando o arquivo que será importado
  uploaded_file = widgets.FileUpload(
      accept='*',
      multiple=False)

  button = widgets.Button(
      description='Enviar',
      button_style='success')

  button.on_click(on_button_clicked)

  display(uploaded_file, button)

In [ ]:
upload = {
    "funcao": "upload_arquivo",
    "parametros": "uploaded_file, button"}

download = {
    "funcao": "download_arquivo",
    "parametros": "filename"}

documents = [upload, download]

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["funcao", "parametros"]
df

In [ ]:
modelo_funcao = "models/embedding-001"
def embed_fn(title, text):
  return genai.embed_content(model=modelo_funcao,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["funcao"], row["parametros"]), axis=1)
df

In [ ]:
def buscar_a_funcao_exibir(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=modelo_funcao,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  if (df.iloc[indice]["funcao"] == 'upload_arquivo'):
    print('Pode agora subir com os seus arquivos!')
    upload_arquivo()
    return 1
  else:
    if (df.iloc[indice]["funcao"] == 'download_arquivo'):
        download_arquivo()
        print('Baixando o arquivo!')
        return 1
    else:
         print('Vou buscar no gemini!')
         return 0


In [ ]:
consulta = "Quero fazer um upload?"

retorno = buscar_a_funcao_exibir(consulta, df, model)
#print(retorno)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                              safety_settings=safety_settings,
                              generation_config=generation_config,)

In [ ]:
chat = model.start_chat(history=[])

In [ ]:
def iniciar_chat():
  texto = input('Digite a sua pergunta: ')

  while texto != "fim":
    if (buscar_a_funcao_exibir(consulta, df, model)) == 1:
       texto = 'fim'
    else :
      response = chat.send_message(texto)
      print(response.text)
      texto = input('Digite a sua pergunta: ')

In [ ]:
iniciar_chat()